In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 debug   #1 run_with_timeout_async / timeout: 60
00:00:00 debug   #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:main@437-6>, Some "C:\home\git\polyglot")
00:00:01 verbose #2 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:01 verbose #3 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 verbose #4 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:01 verbose #5 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:01 debug   #2 run_with_timeout_async / timeout: 500
00:00:01 verbose #3 wait_for_port_access / port: 13805 / retry: 0
00:00:02 verbose #1 sendJson / port:

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00 debug   #1 writeDibCode / output: Spi / path: test.dib
00:00:00 debug   #2 parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00 debug   #1 run_with_timeout_async / timeout: 60
00:00:00 debug   #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64",
        Some <fun:main@437-6>, Some "C:\home\git\polyglot")
00:00:02 verbose #2 > 00:00:00 debug   #1 pwd: C:\home\git\polyglot
00:00:02 verbose #3 > 00:00:00 debug   #2 dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:02 verbose #4 > 00:00:00 debug   #3 targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:02 debug   #2 run_with_timeout_async / timeout: 500
00:00:02 verbose #3 wait_for_port_access / port: 13805 / retry: 0
00:00:02 verbose #5 > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:03 debug   #4 run_with_timeout

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @Nhowka
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> cmd /C dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fable-temp.csproj (in 3.15 sec).
.> cmd /C dotnet restore C:/home/git/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fsproj (in 361 ms).
Project and references (1 source files) parsed in 18851ms

Started Fable compilation...

Fable compilation finished in 2430ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

    Blocking waiting for file lock on package cache
    Blocking waiting for file lock on package cache
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^
    |
    = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:41:8
   |
41 | struct Cart {
   |        ^^^^
   |
   = note: `#[warn(dead_code)]` on by default

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&mut self, item: &Item) {
   |        ^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:124:4
    |
124 | fn parse_comment(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:130:4
    |


In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished `release` profile [optimized] target(s) in 28.67s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_temp_test-27bbac491070caad.exe)

running 3 tests
test test_parse_number ... ok
2024-04-28T12:58:10.494890Z  INFO spiral_temp_test: input=StringLiteral("vrS")
2024-04-28T12:58:10.495027Z  INFO spiral_temp_test: input=Identifier("f6y0RaQa56q4vZc1b011qRJEuLYpT4GZ")
2024-04-28T12:58:10.495112Z  INFO spiral_temp_test: input=Operator(")")
2024-04-28T12:58:10.495178Z  INFO spiral_temp_test: input=Operator(")")
2024-04-28T12:58:10.495291Z  INFO spiral_temp_test: input=Operator(")")
2024-04-28T12:58:10.495374Z  INFO spiral_temp_test: input=StringLiteral("/?<qCq^n),{|/-]")
2024-04-28T12:58:10.495462Z  INFO spiral_temp_test: input=Comment("[`?#/GV.U.`")
2024-04-28T12:58:10.495525Z  INFO spiral_temp_test: input=Comment("ko3:a'/*$g(A^-tg\\}(5X\"x$} ]g\\8j")
2024-04-28T12:58:10.

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
